In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 67960, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 67960 (delta 30), reused 44 (delta 26), pack-reused 67911
Receiving objects: 100% (67960/67960), 576.42 MiB | 32.19 MiB/s, done.
Resolving deltas: 100% (47721/47721), done.


In [2]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1683177 sha256=3c132b5dec54388568f9d16a6e569c9071e1314f995128dd5ed46194deb7ce73
  Stored in directory: /tmp/pip-ephem-wheel-cache-o4mkotm7/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=5b5c671bc65d2a636db55bb714fb6e1350c12335154afe174e38a84440394e21
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=ad62e411c25b3f47bfb0bb2229352d5f28369532cfa2dc7761352eb0412db5bf
  Stored in directory: /root/.cache/pip/wheels/bc/49/5f/fdb5b9d85055c478213e0158ac122b596816149a02d82e0ab1
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78546 sha256=867e75dab2893e4b7e115747ebc0a343a4c3406c916fb40a6

bash: line 1: cd: models/research/: No such file or directory
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
#TEST 
!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-12-14 07:55:45.256202: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W1214 07:55:45.788774 140162670479232 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.66s
I1214 07:55:46.208894 140162670479232 test_util.py:2309] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.66s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.77s
I1214 07:55:46.984534 140162670479232 test

In [5]:
%cd /content/
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip train2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip

!wget http://images.cocodataset.org/zips/test2017.zip
!unzip test2017.zip
!wget http://images.cocodataset.org/annotations/image_info_test2017.zip
!unzip image_info_test2017.zip

Streaming output truncated to the last 5000 lines.
 extracting: test2017/000000541512.jpg  
 extracting: test2017/000000395525.jpg  
 extracting: test2017/000000235435.jpg  
 extracting: test2017/000000069964.jpg  
 extracting: test2017/000000131962.jpg  
 extracting: test2017/000000281392.jpg  
 extracting: test2017/000000534893.jpg  
 extracting: test2017/000000128676.jpg  
 extracting: test2017/000000187639.jpg  
 extracting: test2017/000000153095.jpg  
 extracting: test2017/000000151361.jpg  
 extracting: test2017/000000067484.jpg  
 extracting: test2017/000000358618.jpg  
 extracting: test2017/000000236877.jpg  
 extracting: test2017/000000151014.jpg  
 extracting: test2017/000000351896.jpg  
 extracting: test2017/000000320687.jpg  
 extracting: test2017/000000396067.jpg  
 extracting: test2017/000000318429.jpg  
 extracting: test2017/000000381787.jpg  
 extracting: test2017/000000287482.jpg  
 extracting: test2017/000000459614.jpg  
 extracting: test2017/000000010879.jpg  
 extra

In [6]:
!pip install -U pycocotools

In [7]:
%pylab inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
import tensorflow as tf
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

Populating the interactive namespace from numpy and matplotlib


In [8]:
!git clone https://github.com/philferriere/cocoapi

Cloning into 'cocoapi'...
remote: Enumerating objects: 994, done.
remote: Total 994 (delta 0), reused 0 (delta 0), pack-reused 994
Receiving objects: 100% (994/994), 12.26 MiB | 29.54 MiB/s, done.
Resolving deltas: 100% (589/589), done.


In [77]:
dataDir = '/content'
dataTypeVal = 'val2017'
dataTypeTrain = 'train2017'
dataTypeTest='test2017'
annDir = '{}/annotations'.format(dataDir)

annZipFileTrainVal = '{}/annotations_train{}.zip'.format(dataDir, dataTypeVal)
annZipFileTest='{}/image_info_{}.zip'.format(dataDir, dataTypeTest)

annFileVal = '{}/instances_{}.json'.format(annDir, dataTypeVal)
annFileTrain='{}/instances_{}.json'.format(annDir, dataTypeTrain)
annFileTest='{}/image_info_{}.json'.format(annDir, dataTypeTest)

print (annDir)
print (annZipFileTrainVal)
print (annZipFileTest)

print (annFileTrain)
print (annFileVal)
print (annFileTest)


/content/annotations
/content/annotations_trainval2017.zip
/content/image_info_test2017.zip
/content/annotations/instances_train2017.json
/content/annotations/instances_val2017.json
/content/annotations/image_info_test2017.json


In [78]:
cocotrain=COCO(annFileVal)
imgIds = cocotrain.getImgIds()
print("Total images: {}".format(len(imgIds)))
rand=np.random.randint(0,len(imgIds))
img = cocotrain.loadImgs(imgIds[rand])[0]
print("Image example:")
print(img)
annIds=cocotrain.getAnnIds()
print("\nTotal annotations: {}".format(len(annIds)))


ann=cocotrain.loadAnns(cocotrain.getAnnIds(imgIds=img['id']))
print("Annotation example:")
print(ann[0])


loading annotations into memory...
Done (t=0.54s)
creating index...
index created!
Total images: 5000
Image example:
{'license': 4, 'file_name': '000000322829.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000322829.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-16 20:36:47', 'flickr_url': 'http://farm5.staticflickr.com/4073/4923653485_e2b1549eb2_z.jpg', 'id': 322829}

Total annotations: 36781
Annotation example:
{'segmentation': [[112.36, 312.76, 93.17, 208.58, 104.13, 189.39, 91.34, 159.23, 81.28, 119.93, 109.62, 108.97, 145.26, 67.84, 196.43, 73.32, 252.18, 68.75, 292.38, 63.27, 310.66, 47.74, 328.03, 66.02, 364.58, 71.5, 424.9, 75.15, 451.4, 72.41, 489.78, 88.85, 516.29, 124.5, 540.04, 131.81, 511.72, 199.43, 521.77, 218.62, 498.01, 336.51, 487.95, 334.69, 477.9, 263.41, 441.35, 293.56, 444.09, 238.74, 314.32, 213.14, 164.45, 233.25, 171.76, 288.08, 158.97, 286.25, 134.29, 241.47, 122.41, 304.53, 123.32, 332.86, 107.79, 329.2]], 'area': 72667.89155, 'iscr

In [79]:
cats = cocotrain.loadCats(cocotrain.getCatIds(catIds=[1, 62, 63, 64, 65, 67, 70, 72, 73, 78, 79, 80, 81, 82, 85, 86]))
print("Number of categories: {}".format(len(cats)))
nms=[cat['name'] for cat in cats]
print('\nCOCO categories: \n{}\n'.format(' '.join(nms)))

Number of categories: 16

COCO categories: 
person chair couch potted plant bed dining table toilet tv laptop microwave oven toaster sink refrigerator clock vase



In [80]:
#Create tfrecord output directories
%cd /content/
!mkdir '/content/COCO2017/'
!mkdir '/content/COCO2017/val_tfrecords/'
records_path='/content/COCO2017/val_tfrecords/'
img_dir=('/content/val2017/')
tip='val'

/content
mkdir: cannot create directory ‘/content/COCO2017/’: File exists
mkdir: cannot create directory ‘/content/COCO2017/val_tfrecords/’: File exists


In [63]:

def get_annotationstrain(imgId):
    annIds=cocotrain.getAnnIds(imgIds=imgId)
    anns=cocotrain.loadAnns(annIds)
    segmentations=[]
    segmentation_lengths=[]
    bboxes=[]
    catIds=[]
    iscrowd_list=[]
    area_list=[]
    annotation_ids=[]
    for ann in anns:
        catId=ann['category_id']
        if catId in [1, 62, 63, 64, 65, 67, 70, 72, 73, 78, 79, 80, 81, 82, 85, 86] :
            try:
                catId=ann['category_id']
                bbox=ann['bbox']
                segmentation=ann['segmentation'][0]
                iscrowd=ann['iscrowd']
                area=ann['area']
                annotation_id=ann['id']
            
            except:
                continue
            if((not None in bbox) and (None!=catId)):
                catIds.append(catId)
                segmentations.append(segmentation)
                segmentation_lengths.append(len(segmentation))
                bboxes.append(bbox)
                iscrowd_list.append(iscrowd)
                area_list.append(area)
                annotation_ids.append(annotation_id)

    return len(anns),catIds,segmentation_lengths,sum(segmentations, 0),sum(bboxes, 0),iscrowd_list,area_list,annotation_ids

In [81]:
#Size of each TFRecord file will be 100MB for improving performance
n=len(imgIds)
imgids=imgIds[0:n]
size=0
for i in imgids:
    img=cocotrain.loadImgs(i)
    fn=img[0]['file_name']
    size+=os.path.getsize(img_dir+fn)
avg_size=size/n
limit=int(104857600//avg_size)
total_tfrecords=int(len(imgIds)//limit)
print("{} TFRecord files will be created".format(total_tfrecords))

for i in range(0,total_tfrecords):
    examples=[]
    start=i*limit
    end=start+limit
    imgids=imgIds[start:end]
    
    for img in cocotrain.loadImgs(imgids):
        with open(str(img_dir)+img['file_name'],'rb') as f:
            image_string=f.read()

        objects,catIds,segmentation_lengths,segmentations,bboxes,iscrowd,area,annotation_ids=get_annotationstrain(img['id'])

        # Create a Features message using tf.train.Example.
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_string])),
            'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[img['height']])),
            'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[img['width']])),
            'id': tf.train.Feature(int64_list=tf.train.Int64List(value=[img['id']])),
            'license': tf.train.Feature(int64_list=tf.train.Int64List(value=[img['license']])),
            'file_name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(img['file_name'])])),
            'coco_url': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(img['coco_url'])])),
            'flickr_url': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(img['flickr_url'])])),
            'date_captured': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(img['date_captured'])])),
            #objects-Number of objects in the image
            'objects': tf.train.Feature(int64_list=tf.train.Int64List(value=[objects])),
            #Follwing features hold all the annotations data given for the image
            #category_ids-List of aannotation category ids
            'category_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=catIds)),
            #segmentation_lengths-List of segmentation lengths
            'segmentation_lengths': tf.train.Feature(int64_list=tf.train.Int64List(value=segmentation_lengths)),
            #segmention lists flattened into 1D list
            'segmentations': tf.train.Feature(float_list=tf.train.FloatList(value=segmentations)),
            #bboxes flattened into 1D list
            'bboxes': tf.train.Feature(float_list=tf.train.FloatList(value=bboxes)),
            #List of iscrowd values
            'iscrowd': tf.train.Feature(int64_list=tf.train.Int64List(value=iscrowd)),
            #List of area values
            'area': tf.train.Feature(float_list=tf.train.FloatList(value=area)),
            #List of annotation ids 
            'annotation_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=annotation_ids)),
        }))
        examples.append(example)
    
    with tf.io.TFRecordWriter(records_path+'val_'+str(i)+'.tfrecord') as writer:
        for j in examples:
            writer.write(j.SerializeToString())
    examples.clear()
    print("file {} created".format(i))

7 TFRecord files will be created


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


file 0 created
file 1 created
file 2 created
file 3 created
file 4 created
file 5 created
file 6 created


In [71]:
cocotrain=COCO(annFileTrain)
imgIds = cocotrain.getImgIds()
print("Total images: {}".format(len(imgIds)))
rand=np.random.randint(0,len(imgIds))
img = cocotrain.loadImgs(imgIds[rand])[0]
print("Image example:")
print(img)
annIds=cocotrain.getAnnIds()
print("\nTotal annotations: {}".format(len(annIds)))


ann=cocotrain.loadAnns(cocotrain.getAnnIds(imgIds=img['id']))
print("Annotation example:")
print(ann[0])

loading annotations into memory...
Done (t=21.72s)
creating index...
index created!
Total images: 118287
Image example:
{'license': 3, 'file_name': '000000545317.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000545317.jpg', 'height': 640, 'width': 426, 'date_captured': '2013-11-15 03:00:32', 'flickr_url': 'http://farm3.staticflickr.com/2777/4302254127_9e8fd1f90b_z.jpg', 'id': 545317}

Total annotations: 860001
Annotation example:
{'segmentation': [[350.13, 432.79, 413.27, 460.05, 403.23, 484.45, 381.7, 553.33, 344.39, 566.24, 324.3, 606.42, 321.43, 630.82, 320.0, 640.0, 176.5, 639.43, 173.63, 606.42, 189.42, 571.98, 229.6, 543.28, 261.17, 534.67, 271.21, 528.93, 264.04, 487.32, 255.43, 435.66, 262.6, 411.26, 289.87, 411.26, 301.35, 419.87, 302.78, 445.7, 334.35, 448.57, 347.26, 439.96]], 'area': 29470.2384, 'iscrowd': 0, 'image_id': 545317, 'bbox': [173.63, 411.26, 239.64, 228.74], 'category_id': 70, 'id': 1095114}


In [72]:
cats = cocotrain.loadCats(cocotrain.getCatIds(catIds=[1, 62, 63, 64, 65, 67, 70, 72, 73, 78, 79, 80, 81, 82, 85, 86]))
print("Number of categories: {}".format(len(cats)))
nms=[cat['name'] for cat in cats]
print('\nCOCO categories: \n{}\n'.format(' '.join(nms)))

Number of categories: 16

COCO categories: 
person chair couch potted plant bed dining table toilet tv laptop microwave oven toaster sink refrigerator clock vase



In [70]:
#Create tfrecord output directories
%cd /content/
!mkdir '/content/COCO2017/train_tfrecords/'
records_path='/content/COCO2017/train_tfrecords/'
path_fun='/content/train2017/'
tip='train'
img_dir=('/content/train2017/')

/content
mkdir: cannot create directory ‘/content/COCO2017/train_tfrecords/’: File exists


In [73]:
n=len(imgIds)
imgids=imgIds[0:n]
size=0
for i in imgids:
    img=cocotrain.loadImgs(i)
    fn=img[0]['file_name']
    size+=os.path.getsize(img_dir+fn)
avg_size=size/n
limit=int(104857600//avg_size)
total_tfrecords=int(len(imgIds)//limit)
print("{} TFRecord files will be created".format(total_tfrecords))

for i in range(0,total_tfrecords):
    examples=[]
    start=i*limit
    end=start+limit
    imgids=imgIds[start:end]
    
    for img in cocotrain.loadImgs(imgids):
        with open(str(img_dir)+img['file_name'],'rb') as f:
            image_string=f.read()

        objects,catIds,segmentation_lengths,segmentations,bboxes,iscrowd,area,annotation_ids=get_annotationstrain(img['id'])

        # Create a Features message using tf.train.Example.
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_string])),
            'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[img['height']])),
            'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[img['width']])),
            'id': tf.train.Feature(int64_list=tf.train.Int64List(value=[img['id']])),
            'license': tf.train.Feature(int64_list=tf.train.Int64List(value=[img['license']])),
            'file_name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(img['file_name'])])),
            'coco_url': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(img['coco_url'])])),
            'flickr_url': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(img['flickr_url'])])),
            'date_captured': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(img['date_captured'])])),
            #objects-Number of objects in the image
            'objects': tf.train.Feature(int64_list=tf.train.Int64List(value=[objects])),
            #Follwing features hold all the annotations data given for the image
            #category_ids-List of aannotation category ids
            'category_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=catIds)),
            #segmentation_lengths-List of segmentation lengths
            'segmentation_lengths': tf.train.Feature(int64_list=tf.train.Int64List(value=segmentation_lengths)),
            #segmention lists flattened into 1D list
            'segmentations': tf.train.Feature(float_list=tf.train.FloatList(value=segmentations)),
            #bboxes flattened into 1D list
            'bboxes': tf.train.Feature(float_list=tf.train.FloatList(value=bboxes)),
            #List of iscrowd values
            'iscrowd': tf.train.Feature(int64_list=tf.train.Int64List(value=iscrowd)),
            #List of area values
            'area': tf.train.Feature(float_list=tf.train.FloatList(value=area)),
            #List of annotation ids 
            'annotation_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=annotation_ids)),
        }))
        examples.append(example)
    
    with tf.io.TFRecordWriter(records_path+'train_'+str(i)+'.tfrecord') as writer:
        for j in examples:
            writer.write(j.SerializeToString())
    examples.clear()
    print("file {} created".format(i))

184 TFRecord files will be created


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


file 0 created
file 1 created
file 2 created
file 3 created
file 4 created
file 5 created
file 6 created
file 7 created
file 8 created
file 9 created
file 10 created
file 11 created
file 12 created
file 13 created
file 14 created
file 15 created
file 16 created
file 17 created
file 18 created
file 19 created
file 20 created
file 21 created
file 22 created
file 23 created
file 24 created
file 25 created
file 26 created
file 27 created
file 28 created
file 29 created
file 30 created
file 31 created
file 32 created
file 33 created
file 34 created
file 35 created
file 36 created
file 37 created
file 38 created
file 39 created
file 40 created
file 41 created
file 42 created
file 43 created
file 44 created
file 45 created
file 46 created
file 47 created
file 48 created
file 49 created
file 50 created
file 51 created
file 52 created
file 53 created
file 54 created
file 55 created
file 56 created
file 57 created
file 58 created
file 59 created
file 60 created
file 61 created
file 62 created
fi

In [19]:
%cd models/research/
!sudo python3 setup.py install

/content/models/research
running install
running bdist_egg
running egg_info
creating object_detection.egg-info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
writing manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/exporter_lib_v2.py -> build/lib/object_detection
copying object_detection/model_lib.py -> build/lib/object_detection
copying object_detection/model_tpu_main.py -> build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying

In [20]:
%cd /content/

/content


In [ ]:
!python3 generate_tfrecord.py --csv_input=/content/image_info_test2017.csv --output_path=/content/COCO2017/test_tfrecords/test.record/

In [21]:
%cd /content/
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
# Unzip
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

/content
--2021-12-14 08:26:20--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.195.128, 2607:f8b0:400e:c05::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  --.-KB/s    in 0.1s    

2021-12-14 08:26:20 (137 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coc

In [ ]:
%cd /content/models/research/object_detection/
#train 
!python model_main_tf2.py \
--pipeline_config_path='/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config' \
--model_dir='/content/output_training' --alsologtostderr

In [ ]:
%cd /content/models/research/object_detection
!python exporter_main_v2.py \
--trained_checkpoint_dir=/content/output_training \
--pipeline_config_path=/content/drive/ssd_mobilenet_v2_fpn_320/pipeline.config \
--output_directory /content/inference